# METAR data on nearest airport
https://ourairports.com/data/

https://aviationweather.gov/dataserver/example?datatype=metar

https://aviationweather.gov/adds/dataserver_current/httpparam?dataSource=metars&requestType=retrieve&format=xml&startTime=2023-01-21T19:09:20Z&endTime=2023-01-21T21:09:20Z&stationString=PHTO

In [163]:
# pip install beautifulsoup4
# pip install lxml

In [164]:
import requests
import json
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

In [165]:
### Load airports data
airports = pd.read_csv('airports.csv')
airports = airports[airports.scheduled_service == 'yes'] # only aiprt with weather station

In [166]:
def edit_metars(sd,sh,ed,eh,oaci):
    '''
    Read xml file from https://aviationweather.gov/dataserver/example?datatype=metar API
    and extract METARS.
    sd: start date
    sh: start hour
    ed: end date
    eh: en hour
    returns a pandas dataframe with METAR, temperature, visibility and flight rule
    '''
    
    url = "https://aviationweather.gov/adds/dataserver_current/httpparam?"\
        + "dataSource=metars&requestType=retrieve&format=xml&"\
        +"startTime={}T{}Z&endTime={}T{}Z&stationString={}".format(sd,sh,ed,eh,oaci)
    
    response = requests.get(url)
    bs_data = BeautifulSoup(response.text, "xml")
    # print(bs_data.prettify()) # uncomment to see all xml file
    
    dict_METAR = {}
    for tag in ['raw_text','temp_c','visibility_statute_mi','flight_category']:
        dict_METAR[tag] = []
        for link in bs_data.find_all(tag):
            dict_METAR[tag].append(link.contents[0]) 

    results = pd.DataFrame(dict_METAR)
    results['temp_c'] = results['temp_c'].astype(float)
    results['visibility_statute_mi'] = results['visibility_statute_mi'].astype(float)
    
    return results

In [167]:
lat, lon = 48.403088, 11.517864
sd = '2023-01-21'
sh = '19:09:20'
ed = '2023-01-21'
eh = '21:09:20'

# Get the nearest airport with weather service
best_match = airports.iloc[
    (abs(airports['latitude_deg']-lat) + abs(airports['longitude_deg']-lon)).argsort()
]

# Search valid metars in the 20th nearest aiprorts during the specified time range
for oaci in best_match.head(20).ident.tolist():
    df_metar = edit_metars(sd,sh,ed,eh,oaci)
    if df_metar.shape[0] > 0:
        break

In [168]:
df_metar

,raw_text,temp_c,visibility_statute_mi,flight_category
0,EDDM 212050Z AUTO 32006KT 9999 FEW010 BKN044 M...,-2.0,6.21,VFR
1,EDDM 212020Z AUTO 32004KT 9999 -SN FEW010 BKN0...,-2.0,6.21,MVFR
2,EDDM 211950Z AUTO 33005KT 9000 -SN SCT015 BKN0...,-2.0,5.59,VFR
3,EDDM 211920Z AUTO 33006KT 290V350 6000 -SN FEW...,-2.0,3.73,MVFR
